# Trabalho prático do módulo 01 do Bootcamp de Arquiteto de Big Data

### Importando bibliotecas

In [9]:
# Manipulação e Análise de Dados
import pandas as pd
import numpy as np

# Visualização de Dados
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn import datasets, model_selection, preprocessing, metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Funções de Carregamento e Verificação

#### Carregar Dados

In [10]:
def carregar_dados(caminho):
    """Carrega o arquivo CSV e trata exceções."""
    try:
        df = pd.read_csv(caminho, sep=';')
        print('Arquivo carregado com sucesso!')
        return df
    except FileNotFoundError:
        print('O arquivo não pode ser encontrado.')
    except Exception as e:
        print(f'Ocorreu um erro: {e}')
    return None

#### Verificar dados duplicados

In [11]:
def verifica_dados_duplicados(nome_dataframe, dataframe):
    """Verifica e exibe a quantidade de dados duplicados no dataframe."""
    qtd = dataframe.duplicated().sum()
    print(f'O dataframe {nome_dataframe} possui {qtd} dados duplicados.')

#### Verificar dados ausentes

In [12]:
def verifica_dados_ausentes(nome_dataframe, dataframe):
    """Verifica dados ausentes no dataframe e exibe as colunas com dados ausentes."""
    dados_ausentes = dataframe.isna().sum()
    colunas_com_dados_ausentes = dados_ausentes[dados_ausentes > 0]

    if not colunas_com_dados_ausentes.empty:
        print(f'O dataframe {nome_dataframe} possui dados ausentes.\nColunas com dados ausentes: ')
        print(colunas_com_dados_ausentes)
    else:
        print(f'O dataframe {nome_dataframe} não possui dados ausentes.')

### Coloque o caminho do arquivo

In [13]:
df = carregar_dados('frete_dias_entrega.csv')

Arquivo carregado com sucesso!


### Explorando o dataset

#### Head()

In [14]:
df.head(3)

,Região,Valor do Frete,Dias para Entrega
0,Norte,49.59,20.0
1,Norte,117.57,3.0
2,Norte,98.15,7.0


#### Info()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Região             500 non-null    object 
 1   Valor do Frete     500 non-null    float64
 2   Dias para Entrega  500 non-null    float64
dtypes: float64(2), object(1)
memory usage: 11.8+ KB


#### Valores nulos

In [16]:
df.isnull().sum()

Região               0
Valor do Frete       0
Dias para Entrega    0
dtype: int64

#### Describe()

In [17]:
df.describe()

,Valor do Frete,Dias para Entrega
count,500.000000,500.000000
mean,90.735140,7.512000
std,22.870106,5.733725
min,-5.370000,1.000000
25%,77.107500,3.000000
50%,94.765000,7.000000
75%,107.187500,11.000000
max,128.950000,33.000000


#### Valores nulos

In [18]:
df.isnull().any()

Região               False
Valor do Frete       False
Dias para Entrega    False
dtype: bool

#### Valores duplicados

In [19]:
lista_dataframes = [
    ('frete_dias_entrega', df)
]
for nome, dataframe in lista_dataframes:
    verifica_dados_duplicados(nome, dataframe)

O dataframe frete_dias_entrega possui 1 dados duplicados.


#### Valores ausentes

In [20]:
for nome, dataframe in lista_dataframes:
    verifica_dados_ausentes(nome, dataframe)

O dataframe frete_dias_entrega não possui dados ausentes.


### Tratamento de dados

#### Removendo dados duplicados

In [21]:
df = df.drop_duplicates()

In [23]:
lista_dataframes = [
    ('frete_dias_entrega', df)
]

for nome, dataframe in lista_dataframes:
    verifica_dados_duplicados(nome, dataframe)

O dataframe frete_dias_entrega possui 0 dados duplicados.


### Análise de dados

#### Calculando a média do valor do frete

In [24]:
media_valor_frete = df['Valor do Frete'].mean()
print(f'A média do valor do frete: {media_valor_frete}')

A média do valor do frete: 90.70210420841683


#### Média do valor do frete e dias para entrega por Região

In [25]:
agrupamento_media_regiao = df.groupby('Região').mean()
print(agrupamento_media_regiao)

              Valor do Frete  Dias para Entrega
Região                                         
Centro-Oeste       82.994200           7.710000
Nordeste           91.488000           8.080000
Norte             102.505700           6.950000
Sudeste            87.186500           8.120000
Sul                89.322323           6.757576


#### Filtrar o dataset para a região NORDESTE

In [26]:
df_nordeste = df[df['Região'] == 'Nordeste']

print(df_nordeste.head())

       Região  Valor do Frete  Dias para Entrega
100  Nordeste          120.56                3.0
101  Nordeste          113.59                7.0
102  Nordeste           81.35               12.0
103  Nordeste          106.89                7.0
104  Nordeste           54.58               13.0


#### Média do frete no nordeste

In [27]:
media_frete_nordeste = df_nordeste['Valor do Frete'].mean()
print(media_frete_nordeste)

91.488


### Preparação de Variáveis para o Modelo

#### Criando variável independente

In [28]:
x_valor_frete = df_nordeste.iloc[:,1].values
x_valor_frete

array([120.56, 113.59,  81.35, 106.89,  54.58,  47.74, 124.88, 105.5 ,
        66.4 ,  95.55,  96.02,  99.83,  42.91, 105.37,  98.82, 107.21,
       112.23,  98.09, 104.03,  90.32, 113.86,  47.93, 100.86,  94.84,
       113.21,  29.02, 110.74,  58.57,  99.7 ,  86.48, 118.99,  61.28,
        16.54, 117.94, 103.62, 102.76, 103.23,  98.08, 111.03,  91.7 ,
        85.18,  79.85,  68.18,  87.66,  55.28, 104.86,  39.22,  94.52,
        98.11,  84.58,  89.04,  68.68,  84.51, 105.14, 124.82,  92.44,
       119.18, 113.56, 102.97, 101.04,  95.09, 106.13, 106.8 , 124.53,
        96.4 , 109.02, 114.11,  77.39,  98.04, 116.35,  79.89, 106.49,
        49.52,  50.33, 124.37,  90.46,  79.37,  92.61, 116.13, 113.4 ,
        55.97, 106.79, 103.48,  71.52, 118.57, 109.35,  42.4 ,  92.2 ,
        94.16,  61.04,  96.04, 120.93, 110.64, 104.09,  96.53,  97.6 ,
        65.15,  74.16,  40.08,  90.6 ])

#### Criando variável dependente (O que queremos predizer)

In [29]:
y_dias_entrega = df_nordeste.iloc[:,2].values
y_dias_entrega

array([ 3.,  7., 12.,  7., 13., 20.,  1.,  6., 12., 12.,  7.,  5., 18.,
        1.,  6.,  2.,  3.,  5.,  9.,  9.,  5., 17., 10.,  5.,  1., 18.,
        8., 19.,  9., 10.,  1., 14., 31.,  1.,  7.,  7.,  1.,  4.,  2.,
        4.,  6.,  8., 14.,  8., 17.,  3., 23.,  4.,  9., 14., 12., 17.,
        5.,  3.,  1., 11.,  1.,  1.,  6.,  9., 10.,  4.,  5.,  1.,  5.,
        7.,  3., 13.,  9.,  5., 14.,  3., 14., 16.,  1.,  5., 11.,  6.,
        1.,  6., 12.,  3.,  1., 10.,  1.,  4., 16., 10.,  6., 19.,  5.,
        1.,  1.,  9.,  8.,  7., 15., 11., 15., 11.])

#### Visualizar a correlação

In [30]:
np.corrcoef(x_valor_frete, y_dias_entrega)

array([[ 1.        , -0.89819423],
       [-0.89819423,  1.        ]])

Forte correlação negativa

#### Visualizando o formato da variável

In [31]:
x_valor_frete.shape

(100,)

In [32]:
x_valor_frete = x_valor_frete.reshape(-1,1)
x_valor_frete

array([[120.56],
       [113.59],
       [ 81.35],
       [106.89],
       [ 54.58],
       [ 47.74],
       [124.88],
       [105.5 ],
       [ 66.4 ],
       [ 95.55],
       [ 96.02],
       [ 99.83],
       [ 42.91],
       [105.37],
       [ 98.82],
       [107.21],
       [112.23],
       [ 98.09],
       [104.03],
       [ 90.32],
       [113.86],
       [ 47.93],
       [100.86],
       [ 94.84],
       [113.21],
       [ 29.02],
       [110.74],
       [ 58.57],
       [ 99.7 ],
       [ 86.48],
       [118.99],
       [ 61.28],
       [ 16.54],
       [117.94],
       [103.62],
       [102.76],
       [103.23],
       [ 98.08],
       [111.03],
       [ 91.7 ],
       [ 85.18],
       [ 79.85],
       [ 68.18],
       [ 87.66],
       [ 55.28],
       [104.86],
       [ 39.22],
       [ 94.52],
       [ 98.11],
       [ 84.58],
       [ 89.04],
       [ 68.68],
       [ 84.51],
       [105.14],
       [124.82],
       [ 92.44],
       [119.18],
       [113.56],
       [102.97

In [33]:
x_valor_frete.shape

(100, 1)

### Aplicando os dados ao algoritmo de ML

#### Criando Objeto do tipo Regressão Linear

In [34]:
previsao_valor_frete = LinearRegression()
type(previsao_valor_frete)

sklearn.linear_model._base.LinearRegression

#### Treinando o modelo

In [ ]:
previsao_valor_frete.fit(x_valor_frete, y_dias_entrega)

LinearRegression()

#### Coeficiente Angular

In [37]:
previsao_valor_frete.coef_

array([-0.21979593])

#### Previsão

In [38]:
previsao = previsao_valor_frete.predict(x_valor_frete)
previsao

array([ 1.69009266,  3.2220703 , 10.30829116,  4.69470305, 16.19222827,
       17.69563245,  0.74057423,  5.0002194 , 13.59424035,  7.18718892,
        7.08388483,  6.24646233, 18.7572468 ,  5.02879287,  6.46845622,
        4.62436835,  3.52099277,  6.62890725,  5.32331942,  8.33672165,
        3.1627254 , 17.65387122,  6.02007252,  7.34324403,  3.30559276,
       21.8102123 ,  3.84848871, 15.3152425 ,  6.2750358 ,  9.18073803,
        2.03517227, 14.71959552, 24.55326553,  2.265958  ,  5.41343575,
        5.60246025,  5.49915616,  6.63110521,  3.78474789,  8.03340326,
        9.46647274, 10.63798506, 13.20300359,  8.92137883, 16.03837112,
        5.14088879, 19.56829379,  7.41357873,  6.62451134,  9.5983503 ,
        8.61806044, 13.09310562,  9.61373602,  5.07934593,  0.75376198,
        7.87075427,  1.99341104,  3.22866418,  5.55630311,  5.98050925,
        7.28829505,  4.86174796,  4.71448468,  0.8175028 ,  7.00036238,
        4.22653771,  3.10777642, 11.17868305,  6.63989705,  2.61

#### Avaliando modelo

In [40]:
mae = mean_absolute_error(y_dias_entrega, previsao)
mse = mean_squared_error(y_dias_entrega, previsao)
r2 = r2_score(y_dias_entrega, previsao)
print(f'MAE: {mae}') # mean absolute error
print(f'MSE: {mse}') #
print(f'R²: {r2}')  # score

MAE: 2.1739227562262236
MSE: 6.603950010056614
R²: 0.8067528732689382


#### Coeficiente e Intercepto

In [43]:
# Verificando o coeficiente e intercepto
coeficiente = previsao_valor_frete.coef_
intercepto = previsao_valor_frete.intercept_
print(f'Coeficiente: {coeficiente}')
print(f'Intercepto: {intercepto}')

Coeficiente: [-0.21979593]
Intercepto: 28.188690255366254


### Visualização dos Resultados

In [42]:
grafico = px.scatter(x=x_valor_frete.ravel(), y=y_dias_entrega, title="Valor do Frete vs Dias de Entrega")
grafico.show()
grafico.add_scatter(x=x_valor_frete.ravel(), y=previsao, name='Previsões')
grafico.show()

### Exemplo de Previsão Direta

#### Exemplo 1

In [44]:
# Modo direto
# Passo 1: Criar um array com o valor do frete
valor_frete = np.array([[118.00]])

# Passo 2: Fazer a previsão usando o modelo
previsao_direta = previsao_valor_frete.predict(valor_frete)


# Passo 2: Fazer a previsão usando o modelo
#previsao_dias = previsao.predict(valor_frete)

# Passo 3: Arredondar o resultado
previsao_dias_arredondada = round(previsao_direta[0])
print(f'Previsão de dias para entrega: {previsao_dias_arredondada}')

Previsão de dias para entrega: 2


#### Exemplo 2

In [45]:
# Modo direto
# Passo 1: Criar um array com o valor do frete
valor_frete = np.array([[52.98]])

# Passo 2: Fazer a previsão usando o modelo
previsao_direta = previsao_valor_frete.predict(valor_frete)


# Passo 2: Fazer a previsão usando o modelo
#previsao_dias = previsao.predict(valor_frete)

# Passo 3: Arredondar o resultado
previsao_dias_arredondada = round(previsao_direta[0])
print(f'Previsão de dias para entrega: {previsao_dias_arredondada}')

Previsão de dias para entrega: 17
